# Script to simulate from an OSM file

In [10]:
import os
import sys


if len(sys.argv) < 2:
    print("Usage: provide an osm file name as argument")
    sys.exit(1)

osm_file_name = sys.argv[1]
osm_file_name = 'ixelles'


In [11]:
osm_file_path = os.path.join(os.getcwd(), osm_file_name)
net_file_path = osm_file_path + '.net.xml'
poly_file_path = osm_file_path + '.poly.xml'
trips_file_path = osm_file_path + '.trips.xml'
routes_file_path = osm_file_path + '.routes.xml'
altternative_routes_file_path = routes_file_path[:-4] + ".alt.xml"


In [ ]:
# Network file
os.system(f"netconvert --osm-files {osm_file_path} -o {net_file_path}")

In [ ]:
# TAZ file
os.system(f"polyconvert --net-file {net_file_path} --osm-files {osm_file_path} --type-file $SUMO_HOME/data/typemap/osmPolyconvert.typ.xml -o {poly_file_path}")

In [ ]:
# Trips
beginning_value = 0
end_value = 3600
probability_value = 0.72
os.system(f"python $SUMO_HOME/tools/randomTrips.py -n {net_file_path} -b {beginning_value} -e {end_value} -p {probability_value} -t {poly_file_path} -o {trips_file_path}")

In [ ]:
# Routes
os.system(f"duarouter --net-file {net_file_path} --route-files {trips_file_path} -o {routes_file_path}")

In [ ]:
configuration_file = f"""<configuration>
    <input>
        <net-file value="{net_file_path}"/>
        <route-files value="{routes_file_path}"/>
        <additional-files value="{poly_file_path}"/>
    </input>

    <additional>
        <edgedata-output value="edgeData.xml" />
        <tripinfo-output value="tripinfo.xml" />
    </additional>


</configuration>"""

with open("ixelles.sumocfg", "w") as f:
    f.write(configuration_file)

In [ ]:
print(net_file_path)
print(routes_file_path)
print(altternative_routes_file_path)
print(poly_file_path)

# Analyzing output of simulation

In [2]:
import requests
import os
import xml.etree.ElementTree as ET
import sumolib
import json
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
tree = ET.parse("edgeData.xml")
root = tree.getroot()
edge_data = []

for edge in root.findall('edge'):
    edge_data.append(edge.attrib)

In [4]:
# get all atrributes from root

for child in root.iter("edge"):
     print(child.tag, child.attrib)

edge {'id': '-1001144919', 'sampledSeconds': '3.36', 'traveltime': '3.96', 'overlapTraveltime': '4.55', 'density': '0.21', 'laneDensity': '0.21', 'occupancy': '0.10', 'waitingTime': '0.00', 'timeLoss': '1.76', 'speed': '4.27', 'speedRelative': '0.51', 'departed': '1', 'arrived': '0', 'entered': '0', 'left': '1', 'laneChangedFrom': '0', 'laneChangedTo': '0'}
edge {'id': '-100612596#0', 'sampledSeconds': '9.12', 'traveltime': '1.11', 'overlapTraveltime': '1.99', 'density': '1.16', 'laneDensity': '1.16', 'occupancy': '0.35', 'waitingTime': '0.00', 'timeLoss': '1.40', 'speed': '6.16', 'speedRelative': '0.74', 'departed': '1', 'arrived': '0', 'entered': '4', 'left': '5', 'laneChangedFrom': '0', 'laneChangedTo': '0'}
edge {'id': '-100612596#1', 'sampledSeconds': '5.13', 'traveltime': '0.61', 'overlapTraveltime': '1.28', 'density': '1.01', 'laneDensity': '1.01', 'occupancy': '0.24', 'waitingTime': '0.00', 'timeLoss': '0.39', 'speed': '7.57', 'speedRelative': '0.91', 'departed': '0', 'arrived'

In [5]:
edge_data = []
for edge in root.iter('edge'):
        edge_data.append(edge.attrib)

df = pd.DataFrame(edge_data)
df

,id,sampledSeconds,traveltime,overlapTraveltime,density,laneDensity,occupancy,waitingTime,timeLoss,speed,speedRelative,departed,arrived,entered,left,laneChangedFrom,laneChangedTo
0,-1001144919,3.36,3.96,4.55,0.21,0.21,0.10,0.00,1.76,4.27,0.51,1,0,0,1,0,0
1,-100612596#0,9.12,1.11,1.99,1.16,1.16,0.35,0.00,1.40,6.16,0.74,1,0,4,5,0,0
2,-100612596#1,5.13,0.61,1.28,1.01,1.01,0.24,0.00,0.39,7.57,0.91,0,0,4,4,0,0
3,-100612596#2,50.38,11.95,12.59,0.51,0.51,0.24,0.00,3.67,7.59,0.91,0,0,4,4,0,0
4,-100612596#3,38.80,9.05,9.70,0.53,0.53,0.25,0.00,3.33,7.48,0.90,0,0,4,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,996866375#1,1.46,0.03,0.73,6.74,6.74,0.13,0.00,0.21,7.12,0.85,0,0,2,2,0,0
6987,996866375#2,3.34,1.03,1.67,0.41,0.41,0.12,0.00,0.34,7.48,0.90,0,0,2,2,0,0
6988,998042136#0,1.66,0.04,0.83,7.64,1.91,0.04,0.00,0.35,6.28,0.75,0,0,2,2,0,0
6989,998042136#1,26.42,8.40,9.06,0.43,0.11,0.05,0.00,4.92,6.79,0.82,1,0,2,3,0,0


In [7]:
df["congestion_score"] = df["density"].astype(float) * df["timeLoss"].astype(float)
df["congestion_score"]

0       0.3696
1       1.6240
2       0.3939
3       1.8717
4       1.7649
         ...  
6986    1.4154
6987    0.1394
6988    2.6740
6989    2.1156
6990    4.4253
Name: congestion_score, Length: 6991, dtype: float64

In [9]:
# sort by congestion score

sorted_df = df.sort_values(by=['congestion_score'], ascending=False)
sorted_df

,id,sampledSeconds,traveltime,overlapTraveltime,density,laneDensity,occupancy,waitingTime,timeLoss,speed,speedRelative,departed,arrived,entered,left,laneChangedFrom,laneChangedTo,congestion_score
2190,1174387992#0,236.94,0.04,9.73,266.67,133.33,10.89,201.00,227.62,0.53,0.04,0,0,25,25,0,0,60699.4254
5145,416562592#0,254.24,0.04,23.11,133.33,133.33,23.40,229.00,247.56,0.22,0.02,0,0,11,11,0,0,33007.1748
4980,40506844#9,122.75,0.03,11.20,266.67,133.33,5.64,103.00,115.58,0.46,0.06,1,0,10,11,0,0,30821.7186
3745,258270305#10,1153.74,94.76,96.15,18.05,18.05,8.95,1003.00,1099.12,0.67,0.05,0,0,12,12,0,0,19839.1160
4156,307372336#0,140.33,0.03,15.59,133.33,133.33,12.90,119.00,134.11,0.33,0.04,0,0,9,9,0,0,17880.8863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4656,35830442#8,0.58,0.04,0.58,1.58,1.58,0.05,0.00,0.00,9.19,1.10,0,0,1,1,0,0,0.0000
763,-27807643#0,0.62,0.02,0.62,2.88,2.88,0.06,0.00,0.00,8.33,1.00,0,0,1,1,0,0,0.0000
6449,791118785#2,0.65,0.03,0.65,3.02,3.02,0.06,0.00,0.00,7.95,0.95,0,0,1,1,0,0,0.0000
3244,183454207#3,0.45,0.02,0.45,2.09,0.52,0.01,0.00,0.00,11.50,0.83,0,0,1,1,0,0,0.0000


In [12]:
# read net file
net = sumolib.net.readNet(net_file_path)


In [16]:
# pick the first 20 edges
edges20 = sorted_df["id"].head(20).tolist()
edges20

['1174387992#0',
 '416562592#0',
 '40506844#9',
 '258270305#10',
 '307372336#0',
 '527010849#1',
 '452101333#1',
 '527010849#0',
 '40283535',
 '30240254',
 '117228491#0',
 '307372339#2',
 '13643508#0',
 '124862963#2',
 '23230945#1',
 '760945664#0',
 '40747149#1',
 '583307357#0',
 '258270305#2',
 '5450857#1']

In [17]:
# read xml net file
net = sumolib.net.readNet(net_file_path)
net
    

In [23]:
net.getEdge(edges20[0])

<edge id="1174387992#0" from="3989095435" to="3132141670"/>

In [39]:
tree2 = ET.parse(net_file_path)
root2 = tree2.getroot()



for child in root2:
    if child.tag == "edge" and child.attrib["id"] not in edges20:
        root2.remove(child)
    else:
        root2.remove(child)
   

tree2.write("ixelles20.net.xml")